In [1]:
from settings import *

In [2]:
import pandas as pd

In [3]:
ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

In [ ]:
vessels = pd.read_csv(VESSELS, sep='|')
vessels.head()

In [5]:
# pd.read_csv(SCHEDULES_TO_MAY_2024, sep="|")

In [6]:
ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time'])

In [ ]:
ais_test.head()

In [ ]:
len(ais_test["vesselId"].unique())

In [9]:
# pd.merge(ais_test, vessels, on="vesselId").dropna(subset="length")

In [10]:
# ais_test.sort_values(by=["vesselId", "time"]).iloc[:1]

In [11]:
# ais_test.value_counts(subset="vesselId")

In [12]:
# ais_test[ais_test["vesselId"]=="61e9f38eb937134a3c4bfd8d"]

In [13]:
# ais_test['time'] = pd.to_datetime(ais_test['time'])
# ais_test['time_diff'] = ais_test['time'].diff()

# mean_time_diff = ais_test['time_diff'].mean()
# std_diff = ais_test['time_diff'].std()
# print(mean_time_diff, std_diff)

In [ ]:
ais_test['time'] = pd.to_datetime(ais_test['time'])
ais_test['time_diff'] = ais_test.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()

mean_time_diff_per_id = ais_test.groupby('vesselId')['time_diff'].mean()

print(mean_time_diff_per_id)

In [ ]:
mean_time_diff_per_id.describe()

In [ ]:
ais_test.sort_values(by=['vesselId', 'time'])

In [ ]:
ais_test['time_diff'].describe()

In [ ]:
ais_test[ais_test["vesselId"]=="61e9f38eb937134a3c4bfd8d"]

In [ ]:
ais_test.sort_values(by=["vesselId", "time"]).iloc[240:]

In [ ]:
ais_test["time_diff"].value_counts()

In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram
plt.figure(figsize=(8,6))
plt.hist(mean_time_diff_per_id.dt.total_seconds(), bins=20, edgecolor='black', alpha=0.7)

# Add labels and title
plt.xlabel('Mean Time Difference (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Mean Time Difference Per ID')

# Show the plot
plt.show()

In [22]:

def make_features(df: pd.DataFrame) -> pd.DataFrame:
    raw = df.copy()

    # time_diff: difference of time since last measure
    raw['time'] = pd.to_datetime(raw['time'])
    raw['time_diff'] = raw.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()
    
    
    return raw

In [23]:
ais_tr = make_features(ais_train)

In [ ]:
ais_tr.describe()

In [25]:
te = ais_test.copy()

# time_diff: difference of time since last measure
te['time'] = pd.to_datetime(te['time'])
te['time_diff'] = te.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()

In [ ]:
te.sort_values(by=["vesselId", "time"])

In [ ]:
ais_train

In [ ]:
ais_train.sort_values(by=["vesselId","time"])[["vesselId", "etaRaw", "time"]]

In [ ]:
ais_train["etaRaw"].isna().sum()

In [30]:
lbda = ais_train["etaRaw"].iloc[0]

In [ ]:
lbda.split()

In [33]:
# pd.to_datetime(ais_train["etaRaw"].apply(lambda x: str(x).split()))

In [ ]:
# CREATE time_diff AND MAKE IT IN SECONDS

train_vessel_id_time = ais_train[["vesselId", "time"]]
train_vessel_id_time["split"] = "train"
train_vessel_id_time["ID"] = train_vessel_id_time.index

test_vessel_id_time = ais_test[["ID", "vesselId", "time" ]]
test_vessel_id_time["split"] = "test"
all_times_vesselId = pd.concat([train_vessel_id_time, test_vessel_id_time], ignore_index=True)

all_times_vesselId['time_diff'] = all_times_vesselId.sort_values(by=['time']).groupby("vesselId")['time'].diff().dropna().dt.total_seconds().astype(int)  # .shift(-1)

# arrival time diff (from etaRaw)
# all_times_vesselId['arr_time_diff'] = all_times_vesselId.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff().shift(-1)

# ais_test["time_diff"] = all_times_vesselId[all_times_vesselId["split"]=="test"].sort_values(by="ID").reset_index()["time_diff"]
# ais_train["time_diff"] = all_times_vesselId[all_times_vesselId["split"]=="train"].sort_values(by="ID").reset_index()["time_diff"]

# nb_dt_na_test = ais_test["time_diff"].isna().sum()
# ais_test["time_diff"] = ais_test.sort_values(by=["time_diff"]).iloc[:-nb_dt_na_test]["time_diff"].dt.total_seconds().astype(int)

# nb_dt_na_train = ais_train["time_diff"].isna().sum()
# ais_train["time_diff"] = ais_train.sort_values(by=["time_diff"]).iloc[:-nb_dt_na_train]["time_diff"].dt.total_seconds().astype(int)

In [88]:
ais_train["time_diff"] = all_times_vesselId[all_times_vesselId["split"]=="train"]["time_diff"] # .dt.total_seconds().astype(int) # .sort_values(by="ID")
ais_test["time_diff"] = all_times_vesselId[all_times_vesselId["split"]=="test"].reset_index()["time_diff"]

In [ ]:
ais_test.sort_values(by=["vesselId","time"])

In [ ]:
ais_test.isna().sum()

In [97]:
ais_train.dropna(subset="time_diff", inplace=True)

In [ ]:
# from datetime import datetime

# dt1 = datetime.fromisoformat("2024-05-08 00:12:27")
# dt2 = datetime.fromisoformat("2024-05-08 00:39:27")
# (dt2 - dt1).total_seconds()

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

input_features = ["vesselId", "time",
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]
output_features = ["vesselId", "time",
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]


In [193]:
scaler = StandardScaler()

In [194]:
df_train = ais_train.drop(columns="portId")

In [ ]:
X = df_train.sort_values(by=["vesselId", "time"]) # [input_features]
# X = X.groupby("vesselId").apply(lambda x: x.sort_values(by="time").iloc[:-1]).reset_index(drop=True)[input_features]

y = df_train.sort_values(by=["vesselId", "time"]) # [input_features]
# y = y.groupby("vesselId").apply(lambda x: x.sort_values(by="time").iloc[1:]).reset_index(drop=True)[output_features]

In [217]:
X = X.drop(columns=["vesselId", "time"])
y = y.drop(columns=["vesselId", "time"])

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, shuffle=False)

In [220]:
import xgboost as xgb

In [227]:
params = {
    'n_estimators': 5000,
    'gamma': 0.5,
    'subsample': 0.6,
    'n_estimators': 5000,
    'min_child_weight':  15,
    'colsample_bytree': 0.8,
    'max_depth': 4,
    'eta':  0.005,
    'refresh_leaf': 1,
}

In [231]:
xgb_model = xgb.XGBRegressor(**params)

In [ ]:
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    # early_stopping_rounds=50,
    verbose=False,
)

In [ ]:
results = xgb_model.evals_result()
train_errors = results['validation_0']['rmse']
test_errors = results['validation_1']['rmse']

plt.figure(figsize=(10, 6))
plt.plot(train_errors, label='Train')
plt.plot(test_errors, label='Test')
plt.xlabel('Boosting Rounds')
plt.ylabel("rmse")
plt.legend()
plt.title('Learning Curves')
plt.show()

In [ ]:
xgb_model.predict(X_test) 

In [ ]:
y_test

In [ ]:
ais_test.iloc[-100:]